In [1]:
import os
import json


In [2]:
from transformers import RobertaModel, RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [27]:
dataset = "../../RELiC"
max_len = 512
book_wise_stats = {}

In [30]:
def getTokensLen(text):
    tokenized = tokenizer(text)
    #print(len(tokenized["input_ids"]))
    print(len(tokenized["input_ids"]))
    return len(tokenized["input_ids"])


In [37]:
def calculateContextExpansion():
    train_file = os.path.join(dataset, "train.json")
    with open(train_file) as file:
        train_data = json.load(file)
    for book in train_data:
        sentences = train_data[book]["sentences"]
        book_wise_stats[book] = {"old_stats" : [], "new_stats" : []}
        print("Completed: ", book)
        for quoteId in train_data[book]["quotes"]:
            quoteStart = train_data[book]["quotes"][quoteId][1]
            quoteStartOld = quoteStart
            quoteLen = train_data[book]["quotes"][quoteId][2]
            quote = " ".join(sentences[quoteStart:quoteStart+quoteLen])
            quoteTokenLen = getTokensLen(quote)
            flipDigit = -1
            book_wise_stats[book]["old_stats"].append(quoteLen)
            
            while quoteTokenLen < max_len and quoteStart >= 0:
                #print(sentences[quoteStart:quoteStart+quoteLen])
                #print(" came")
                # can expand context. Expand preceding and succeeding alternatively.
                if flipDigit < 0:
                    quoteStart -= 1
                quoteLen += 1
                quote = " ".join(sentences[quoteStart:quoteStart+quoteLen])
                quoteTokenLen = getTokensLen(quote)
                flipDigit = flipDigit * -1
            book_wise_stats[book]["new_stats"].append(quoteLen + quoteStartOld - quoteStart)
            #break
        #break
            
calculateContextExpansion()

Completed:  brothers_karamazov
Completed:  to_the_lighthouse
Completed:  the_pickwick_papers
Completed:  david_copperfield
Completed:  animal_farm
Completed:  the_scarlet_letter
Completed:  a_portrait_of_the_artist_as_a_young_man
Completed:  the_turn_of_the_screw
Completed:  the_souls_of_black_folk
Completed:  adam_bede
Completed:  sense_and_sensibility
Completed:  martin_chuzzlewit
Completed:  swanns_way
Completed:  sister_carrie
Completed:  daisy_miller
Completed:  o_pioneers
Completed:  the_red_badge_of_courage
Completed:  little_dorrit
Completed:  great_expectations
Completed:  the_call_of_the_wild
Completed:  mrs_dalloway
Completed:  the_sport_of_the_gods
Completed:  middlemarch
Completed:  alices_adventures_in_wonderland
Completed:  jacobs_room
Completed:  1984
Completed:  house_of_mirth
Completed:  nicholas_nickleby
Completed:  moby_dick
Completed:  oliver_twist
Completed:  jane_eyre
Completed:  this_side_of_paradise
Completed:  madame_bovary
Completed:  iola_leroy
Completed:  f

In [38]:
print(book_wise_stats)

{'brothers_karamazov': {'old_stats': [1, 3, 2, 4, 2, 1, 1, 3, 3, 3, 3, 2, 2, 1, 1, 2, 1, 1, 5, 1, 3, 1, 1, 1, 1, 3, 4, 1, 1, 2, 2, 2, 3, 2, 1, 1, 1, 1, 1, 2, 4, 1, 2, 2, 5, 1, 1, 1, 1, 1, 2, 1, 1, 3, 5, 1, 2, 2, 3, 1, 1, 5, 2, 1, 1, 4, 5, 1, 2, 5, 2, 1, 2, 1, 1, 1, 1, 3, 5, 1, 2, 1, 2, 1, 1, 1, 1, 5, 1, 1, 1, 2, 2, 5, 2, 2, 3, 3, 2, 1, 1, 2, 1, 3, 5, 4, 3, 2, 4, 1, 1, 4, 1, 2, 3, 1, 1, 1, 1, 2, 2, 1, 4, 2, 2, 3, 5, 1, 5, 2, 1, 1, 1, 4, 2, 2, 2, 1, 1, 1, 2, 1, 4, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 5, 3, 1, 2, 2, 1, 2, 1, 3, 1, 3, 2, 3, 1, 4, 1, 1, 1, 2, 1, 2, 4, 5, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 4, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 4, 1, 2, 3, 1, 2, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 5, 1, 2, 1, 4, 1, 1, 1, 1, 1, 1, 2, 2, 5, 1, 1, 3, 1, 3, 1, 1, 1, 5, 3, 1, 2, 3, 1, 1, 3, 5, 5, 1, 1, 1, 4, 3, 3, 1, 1, 2, 1, 3, 3, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 3, 2, 1, 3, 1, 1, 3, 1, 5, 1, 1, 1, 1, 5, 5, 4, 2, 1, 2, 3, 1, 1, 1, 2, 1, 3, 5, 3, 4, 2, 2, 2, 3, 1, 3, 1, 1, 1, 1, 5, 2, 1, 1, 4,

In [39]:
for book in book_wise_stats:
    book_wise_stats[book]["old_stats"] = sum(book_wise_stats[book]["old_stats"])/len(book_wise_stats[book]["old_stats"])
    book_wise_stats[book]["new_stats"] = sum(book_wise_stats[book]["new_stats"])/len(book_wise_stats[book]["new_stats"])


In [40]:
print(book_wise_stats)

{'brothers_karamazov': {'old_stats': 2.0180722891566263, 'new_stats': 41.12267250821468}, 'to_the_lighthouse': {'old_stats': 2.268159475696341, 'new_stats': 37.343528126706715}, 'the_pickwick_papers': {'old_stats': 2.129496402877698, 'new_stats': 32.03309352517986}, 'david_copperfield': {'old_stats': 2.168280871670702, 'new_stats': 33.25968523002421}, 'animal_farm': {'old_stats': 2.167883211678832, 'new_stats': 39.113138686131386}, 'the_scarlet_letter': {'old_stats': 2.057530704589528, 'new_stats': 27.819650937297997}, 'a_portrait_of_the_artist_as_a_young_man': {'old_stats': 2.1245729303547964, 'new_stats': 40.366622864651774}, 'the_turn_of_the_screw': {'old_stats': 2.044, 'new_stats': 39.9184}, 'the_souls_of_black_folk': {'old_stats': 2.3520179372197307, 'new_stats': 28.7847533632287}, 'adam_bede': {'old_stats': 2.131868131868132, 'new_stats': 28.8989010989011}, 'sense_and_sensibility': {'old_stats': 2.2213333333333334, 'new_stats': 31.672}, 'martin_chuzzlewit': {'old_stats': 2.211320